In [3]:
# Continuous Pendulum with DDPG

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from DDPG_Agent import DDPG_Agent
import gym

settings = {
    'num_state': 3,
    'num_action':1,
    'DDPG':{
        'USECONV':False,
        'learning_rate_A':0.01,
        'learning_rate_C':0.01,
        'gamma':0.99,
        'batch_size':32,
        'replay_buffer_size':5000,
        'soft_update_rate':0.1,
        'num_fc1':20,
    }
}

In [4]:
# test with pendulum
env = gym.make('Pendulum-v0')
tf.reset_default_graph()
sess = tf.Session()
RL = DDPG_Agent(settings, sess)
sigma = 3
rlog = []
for episode in range(500):
    s = env.reset()
    rsum = 0
    for step in range(200):
        action = sess.run(RL.A, feed_dict={RL.S:[s]})[0]
        action = [np.clip(np.random.normal(action, sigma), -1, 1)]
        s1, r, d, info = env.step(action)
        rsum += r
        RL.buffer_add(s, action, r, s1, d)
        if RL.buffer_ready:
            RL.train_it()
            sigma *= 0.99995
        if d: break
        s = s1
    # End of episode
    rlog.append(rsum)
    if RL.buffer_ready:
        print 'episode:', episode, 'reward:', rsum, sigma

[2017-07-27 22:43:32,807] Making new env: Pendulum-v0


episode: 1 reward: -1016.22840257 2.98817307787
episode: 2 reward: -1207.16716338 2.95843951933
episode: 3 reward: -1269.51087902 2.92900182199
episode: 4 reward: -1742.20765909 2.89985704193
episode: 5 reward: -1636.23370012 2.87100226448
episode: 6 reward: -967.039368502 2.84243460401
episode: 7 reward: -1087.05941913 2.81415120358
episode: 8 reward: -1307.80664755 2.78614923469
episode: 9 reward: -1285.38179781 2.75842589698
episode: 10 reward: -1435.80547323 2.73097841794
episode: 11 reward: -1037.24654797 2.70380405268
episode: 12 reward: -1380.74427675 2.6769000836
episode: 13 reward: -1081.17515288 2.65026382015
episode: 14 reward: -1422.03296545 2.62389259853
episode: 15 reward: -1824.63547509 2.59778378148
episode: 16 reward: -1413.08664528 2.57193475797
episode: 17 reward: -934.539846998 2.54634294293
episode: 18 reward: -1622.39251324 2.52100577705
episode: 19 reward: -1372.63832078 2.49592072645
episode: 20 reward: -1789.06422246 2.47108528248
episode: 21 reward: -1404.9531

KeyboardInterrupt: 

In [ ]:
plt.plot(rlog)
# demostration
for episode in range(1000):
    s = env.reset()
    for step in range(200):
        action = sess.run(RL.A, feed_dict={RL.S:[s]})[0]
        s, r, d, info = env.step(action)
        env.render()
        if d: break
    